In [ ]:
import pandas as pd
df_resp = pd.read_csv('/project2/wparker/RCLIF_respiratory_support_09282023.csv')
df_resp.head()

In [ ]:
#get just invasive or non-invasive mechanical ventilation
vent = df_resp.loc[(df_resp['device_name']=='Vent') | (df_resp['device_name']=='NIPPV')]
vent['recorded_time'] = pd.to_datetime(vent['recorded_time'], format='%Y-%m-%d %H:%M:%S')

# minimum time by person

vent_grouped = vent.groupby('C19_HAR_ID')['recorded_time'].agg(['min'])
vent_grouped = vent_grouped.rename(columns={'min': 'min_resp_support_time'})
vent_grouped.head()


In [ ]:
df_meds = pd.read_csv('/project2/wparker/RCLIF_meds_admin_conti.csv')
df_meds.head()

In [ ]:
pressors = df_meds.loc[(df_meds['med_name']=='phenylephrine') | 
                       (df_meds['med_name']=='epinephrine') | 
                       (df_meds['med_name']=='vasopressin') | 
                       (df_meds['med_name']=='dopamine') |
                       (df_meds['med_name']=='dobutamine') |
                       (df_meds['med_name']=='norepinephrine') |
                       (df_meds['med_name']=='angiotensin') |
                       (df_meds['med_name']=='isoproterenol')]
pressors = pressors[["C19_HAR_ID", "admin_time"]]
print(len(pressors))
pressors_no_na = pressors.dropna()
print(len(pressors_no_na))
pressors_no_na['admin_time'] = pd.to_datetime(pressors_no_na['admin_time'], format='%Y-%m-%d %H:%M:%S')


pressors_grouped = pressors_no_na.groupby('C19_HAR_ID')['admin_time'].agg(['min'])
pressors_grouped = pressors_grouped.rename(columns={'min': 'min_pressor_time'})
pressors_grouped.head()

In [ ]:
df = pressors_grouped.join(vent_grouped, on='C19_HAR_ID', how='outer')


In [ ]:
df = df[['C19_HAR_ID','min_pressor_time','min_resp_support_time']]
df['min_pressor_time'] = pd.to_datetime(df['min_pressor_time'], format='%Y-%m-%d %H:%M:%S')
df['min_resp_support_time'] = pd.to_datetime(df['min_resp_support_time'], format='%Y-%m-%d %H:%M:%S')


df['life_support_start'] = df[['min_pressor_time','min_resp_support_time']].min(axis=1)


In [ ]:
df = df[['C19_HAR_ID', 'life_support_start']]
display(df)

In [ ]:
from pathlib import Path 
filepath=Path('/project2/wparker/cohort.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 

df.to_csv(filepath, index=False)